# Gapfill a draft GEM using the `dnngior`gapfiller

# Installation

To run the `dnngior` gapfiller we need to have the [Gurobi](https://www.gurobi.com) solver. 

In [22]:
!pip install -i https://pypi.gurobi.com gurobipy

To run `gurobi`, you first need a [license](https://www.gurobi.com/downloads/). 
If you are an acedemic, you may get a license for free.


> **Attention!** To run `dnngior`, the Gurobi solver is **not** optional! 
Without a valid installation of Gurobi, `dnngior` will fail.

Once you have successfully got `gurobi`, you are ready to install the `dnngior` gapfiller. 

In [3]:
!pip install dnngior

You are now ready to gapfill any draft GEM as long as it's using the ModelSEED ontology for compounds and reactions.

## Gapfill using a complete medium

Let us get a genome scale reconstruction of a *Blautia* strain to work with:

In [42]:
import os, sys

cwd         = os.getcwd()
repo_path   = "/".join(os.path.abspath(cwd).split("/")[:-1])
models_path = os.path.join(repo_path, "docs/models")
path_to_blautia_model = os.path.join(models_path, "bh_ungapfilled_model.sbml")

Now, we can load the initial reconstruction, check its medium and whether it grows on it. So. let's print the first 4 exchange reactions on the current medium and see how many exchange reactions are present on the initial reconstruction.

In [79]:
import cobra
draft_reconstruction = cobra.io.read_sbml_model(path_to_blautia_model)

# Check current medium
counter = 0 
for i in model.medium:
    counter += 1
    if counter < 5:
        print(draft_reconstruction.reactions.get_by_id(i).name)
print("Number of compounds on initial medium: ", len(draft_reconstruction.medium))


'' is not a valid SBML 'SId'.


Exchange for H+ [e] [e0]
Exchange for Adenosine [e] [e0]
Exchange for Fumarate [e] [e0]
Exchange for Nitrite [e] [e0]
Number of compounds on initial medium:  148


Now let us see if the initial reconstruction is a growing model. 

In [44]:
draft_reconstruction.optimize()  

,fluxes,reduced_costs
rxn02201_c0,0.0,0.0
rxn00351_c0,0.0,0.0
rxn00836_c0,0.0,0.0
rxn05318_c0,0.0,0.0
rxn01094_c0,0.0,0.0
...,...,...
SK_cpd02701_c0,0.0,0.0
SK_cpd11416_c0,0.0,0.0
SK_cpd15302_c0,0.0,0.0
SK_cpd03091_c0,0.0,0.0


Apparently, this reconstruction is not growing.

So, let's gapfill it! 

Import the `dnngior` library and use the `Gapfill` class to gapfill the reconstruction. 

In [45]:
import dnngior
gapfill_complete_medium = dnngior.Gapfill(draftModel = path_to_blautia_model, 
                                          medium = None, 
                                          objectiveName = 'bio1')

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


Loading network at user provided path
Converting to binary array:
#reactions not found in keys:  155 / 1656


/home/luna.kuleuven.be/u0156635/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  73622 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  36811 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  18405 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9202 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4601 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2300 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  1150 


Warning for adding constraints: z

Get the number of reactions added during the gapfilling along with their names. 

In [46]:
print("Number of eactions added:", len(gapfill_complete_medium.added_reactions))
print("~~")
for reaction in gapfill_complete_medium.added_reactions:
    print(gf_model_compl_med.reactions.get_by_id(reaction).name)

Number of eactions added: 17
~~
murein polymerizing transglycosylase
R_R05617_c
EX_cpd15432_e0
acyl-CoA:malonyl-CoA C-acyltransferase (decarboxylating, oxoacyl- and enoyl-reducing, thioester-hydrolysing)
-
acyl-CoA:malonyl-CoA C-acyltransferase (decarboxylating, oxoacyl-
S-adenosyl-L-methione:demethylmenaquinone methyltransferase
EX_cpd15511_e0
stearoyl-cardiolipin synthase
glycolaldehyde:NAD+ oxidoreductase
glycine/sarcosine/dimethylglycine N-methyltransferase
EX_cpd02557_e0
2-Amino-4-hydroxy-6-(erythro-1,2,3-trihydroxypropyl) dihydropteridine triphosphate phosphohydrolase (alkaline optimum)
all-trans-Octaprenyl diphosphate
EX_cpd02229_e0
core oligosaccharide lipid A transport via vector (periplasm to extracellular)



Now make a new object of the gapfilled model.

In [47]:
gf_model_compl_med = gapfill_complete_medium.gapfilledModel.copy()

Read LP format model from file /tmp/tmp1cme7kgq.lp
Reading time = 0.01 seconds
: 1402 rows, 3346 columns, 14424 nonzeros


And check whether it grows on its medium.

In [48]:
gf_model_compl_med.optimize()

,fluxes,reduced_costs
EX_cpd01329_e0,0.000000,0.0
rxn04070_c0,0.432545,0.0
rxn05467_c0,-1000.000000,0.0
rxn00543_c0,0.000000,-0.0
rxn00527_c0,0.000000,0.0
...,...,...
rxn00131_c0,-41.057192,0.0
rxn05682_c0,0.000000,0.0
rxn00184_c0,0.000000,0.0
rxn01241_c0,0.000000,0.0


Check what compounds are now present on the model's medium that were not in the initial's reconstruction.

In [82]:
check = False
for i in draft_reconstruction.medium:
    if i not in gf_model_compl_med.medium:
        print("Compound ", i , " was part of the initial's model model but not in the gapfilled.")
        check = True
if not check:
    print("All exchange reactions of the initial reconstruction are present on the gapfilled model.\n")

for j in gf_model_compl_med.medium:
    if j not in draft_reconstruction.medium:
        print("Compound ", gf_model_compl_med.metabolites.get_by_id(j[3:]).name, " has been added in the gapfilled model's medium.")

All exchange reactions of the initial reconstruction are present on the gapfilled model.

Compound  Bactoprenyl diphosphate_e0  has been added in the gapfilled model's medium.
Compound  Farnesylfarnesylgeraniol_e0  has been added in the gapfilled model's medium.
Compound  two linked disacharide pentapeptide murein units (uncrosslinked, middle of chain)_e0  has been added in the gapfilled model's medium.
Compound  core oligosaccharide lipid A_e0  has been added in the gapfilled model's medium.


## Gapfill using a defined medium

Load the media file. 

In [50]:
media_file = os.path.join(repo_path, 'docs/biochemistry/Nitrogen-Nitrite_media.tsv')

In [60]:
import pandas as pd
raw_data = pd.read_csv(media_file, sep="\t")
raw_data.head()

,id,name,concentration,minflux,maxflux
0,cpd00027,D-Glucose,0.001,-100,5
1,cpd00075,Nitrite,0.001,-100,5
2,cpd00009,Phosphate,0.001,-100,5
3,cpd00048,Sulfate,0.001,-100,5
4,cpd00063,Ca2+,0.001,-100,100


Load the new media to a dictionary setting the exchange reaction as a key and a dictionary as its value, including a lower, an upper bound and the stoichiometry of the related compound.

In [62]:
Nit_media = {}
with open(media_file) as f:
    f.readline()
    for line in f:
        a = line.strip().split('\t')
        Nit_media['EX_' + a[0] + '_e0'] = {'lower_bound':-1, 'upper_bound':1, 'metabolites':{a[0]+'_e0':-1.0}}

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


Loading network at user provided path
Converting to binary array:
#reactions not found in keys:  7 / 1508


/home/luna.kuleuven.be/u0156635/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  71554 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  35777 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  17888 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  8944 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4472 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2236 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  1118 


Warning for adding constraints: z

Gapfill the initial reconstruction using the nitrate media.

In [63]:
gapfill_nitr     = dnngior.Gapfill(path_to_blautia_model, medium = Nit_media, objectiveName = 'bio1')

'' is not a valid SBML 'SId'.
'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


Loading network at user provided path
Converting to binary array:
#reactions not found in keys:  7 / 1508


/home/luna.kuleuven.be/u0156635/.local/lib/python3.10/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  71554 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  35777 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  17888 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  8944 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4472 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2236 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  1118 


Warning for adding constraints: z

Again, make an object of the gapfilled model and check whether it grows.

In [65]:
gf_model_Nit_med = gapfill_nitr.gapfilledModel.copy()
gf_model_Nit_med.optimize()

Read LP format model from file /tmp/tmp_99qbd2m.lp
Reading time = 0.00 seconds
: 1419 rows, 3166 columns, 14758 nonzeros


,fluxes,reduced_costs
rxn04070_c0,0.224345,0.0
rxn05467_c0,0.000000,0.0
rxn00543_c0,0.000000,0.0
rxn00527_c0,-0.081221,0.0
rxn08287_c0,5.506151,0.0
...,...,...
rxn00131_c0,11.392259,0.0
rxn05682_c0,0.000000,0.0
rxn00184_c0,0.000000,0.0
rxn01241_c0,0.000000,0.0


In [84]:
print("Number of eactions added:", len(gapfill_nitr.added_reactions))
print("~~")
for reaction in gapfill_nitr.added_reactions[:5]:
    print(gf_model_Nit_med.reactions.get_by_id(reaction).name)

Number of eactions added: 75
~~
Hydrogen selenide:NADP+ oxidoreductase
Deoxyadenosine 5'-triphosphate:DNA deoxynucleotidyltransferase (DNA-directed)
Protein synthesis (B. subtilis)
R_R05617_c
EX_cpd10515_e0


The gapfilled model is growing again! 
However it's optimal objective value is lower and the number of the exchange reactions added is rather higher. 


## Build a GEM with `RAST-tk` and `ModelSEEDpy`

Here we show how one can get a GEM programmatically (no GUI) using the `ModelSEEDpy` library. 
This step will provide an input reconstruction for the `dnngior` gapfiller. 
This example is only instructive and is not guaranteed by the authors to work since it depends on third party software.

Starting from a genome, the user needs to annotate it first before building a GEM.
A good option for this step is the [BV-BRC Command Line Interface](https://www.bv-brc.org/docs//cli_tutorial/cli_installation.html) that allows to RAST-annotate the genome.

Once you install the CLI, you may go for the default RASTtk Pipeline. Here is how we annotated a [*Salmonella* genome](https://www.ncbi.nlm.nih.gov/assembly/GCF_019918175.1) found in GenBank.

First, initiate a RAST genome using the `.fna` file of yours. 

In [ ]:
!rast-create-genome --scientific-name "Salmonella infantis" \
                    --genetic-code 11\
                    --domain Bacteria\
                    --contigs GCF_019918175.1_ASM1991817v1_genomic.fna\
                    --genome-id S_infantis_ASM1991817v1\
                    --ncbi-taxonomy-id 595\
                    --source https://www.ncbi.nlm.nih.gov/assembly/GCF_019918175.1\
                    --source-id ASM1991817v1 > S_infantis.gto

>**Attention!** In case you have any trouble when running the `rast`- commands, please consider checking the status of the [RAST server](https://rast.nmpdr.org/rast.cgi). As already mentioned, all steps described in this section have nothing to do with the `dnngior` library, so for any issues you need to contact either the RAST or the ModeSEED communities.

Then run the default RASTtk pipeline tool.

In [ ]:
!rast-process-genome < S_infantis.gto > S_infantis.gto2

Finally, export your annotation in a valid format for the `ModelSEEDpy` library.

In [69]:
!rast-export-genome protein_fasta < S_infantis.gto2 > S_infantis.faa

malformed JSON string, neither tag, array, object, number, string or atom, at character offset 0 (before "(end of string)") at /usr/share/bvbrc-cli/deployment/lib/Bio/KBase/GenomeAnnotation/CmdHelper.pm line 230, <STDIN> line 1.


Once you have your annotated genome, you are ready to go for the reconstruction step.

So first, you need to get the `ModelSEEDpy` library

In [66]:
!git clone https://github.com/ModelSEED/ModelSEEDpy
!pip install ModelSEEDpy/.

Cloning into 'ModelSEEDpy'...
remote: Enumerating objects: 3398, done.
remote: Counting objects: 100% (697/697), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 3398 (delta 548), reused 563 (delta 508), pack-reused 2701
Receiving objects: 100% (3398/3398), 8.28 MiB | 8.91 MiB/s, done.
Resolving deltas: 100% (2387/2387), done.
Defaulting to user installation because normal site-packages is not writeable
Processing ./ModelSEEDpy
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached pyeda-0.2

  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Created wheel for ModelSEEDpy: filename=ModelSEEDpy-0.3.1-py3-none-any.whl size=155900 sha256=65321658f4de7c405423d85405f9d6e9038e228f0d91fb90a76143e1d4fbec7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qzxdwpcc/wheels/ae/94/c7/ac0c2d893134945908b87ad59d3c4553ae69ca45a733271b6e
  Created wheel for chemw: filename=ChemW-0.3.5-py3-none-any.whl size=480112 sha256=e4df06b1f2f087ff0391498b0eff8380df17e78012387809529ef8ae1e778a38
  Stored in directory: /home/luna.kuleuven.be/u0156635/.cache/pip/wheels/7b/21/ba/2dd9467b94a05e280ee3881b019af128e862c92fd4013b34a5
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13834 sha256=acaabc2bd8c97f0ce4b2a60e8305133340379ee18cd0a3077c81036147eba42b
  Stored in directory: /home/luna.kuleuven.be/u0156635/.cache/pip/wheels/84/45/0e/b597debba098119b642eaf728ae1883d23ad8ea2a9366f2ded
  Created wheel for pyeda: filename=pyeda-0.28.0-cp39-cp39-linux_x86_64.whl

In [ ]:
from modelseedpy import MSBuilder, MSGenome
import cobra

# Set the path to your annotation
genome_annotation = "S_infantis.faa"
# Set modelId
modelId  = "s_inf"
# Init a MSGenome instance 
msGenome = MSGenome.from_fasta(genome_annotation, split=' ')
# Build your GEM
model = MSBuilder.build_metabolic_model(model_id = modelId, 
                                        genome = msGenome, 
                                        index= "0", 
                                        classic_biomass = True, 
                                        gapfill_model = False, 
                                        gapfill_media = None, 
                                        annotate_with_rast = True, 
                                        allow_all_non_grp_reactions = True)
# Save the GEM as a .sbml file
modelname = "s_inf_modelseed.sbml"
cobra.io.write_sbml_model(cobra_model = model, filename = modelname)


Now, you can use the draft reconstruction (`s_inf_modelseed.sbml`) as input for the `dnngior` library to gapfill it as you wish! &#x1F389;

### Gapfilig a model with different costs for reactions

Now we need to provide a python dictionary with reaction costs. The function will automatically give a cost of zero to reactions that are already in the draft model and also to the exchange reactions of the defined media. It will give a default cost (set to 1) to reactions that are not in the draft model nor in the dictionary with costs.

In [25]:
# #select random reaction and add random costs between 0 and 1
# candidate_reactions = {}
# for i in  all_reactions.reactions:
#     if np.random.uniform()<0.02:
#         candidate_reactions[i] = np.random.uniform()

        
#use NN scores        
s_file = open(os.path.join(path.parent, 'files', 'custom scores','prediction_example.json'))
NN_scores = json.loads(s_file.read())
s_file.close()



candidate_reactions = {}
for i in all_reactions.reactions:
    if i in NN_scores.keys():
        candidate_reactions[i] = NN_scores[i]


We will also use a different criteria to choose the gapfilled reaction. Instead of 'min_reactions' we use 'min_cost', selecting the set with a minimum sum of costs.

### Quick idea of what to do when no flux is observed on a particular media


1) Run a gapfill on the with complete media option (as in [5])

2) Run flux variability analysis (for big models this takes a long time to run):

In [31]:
model_exchanges = [i for i in model.reactions if ('EX_' in i.id) and ('_e0' in i.id)]

In [ ]:
fva = cobra.flux_analysis.flux_variability_analysis(model, model_exchanges)
print(fva)

The exchange reaction where the minimum and maximum are negative are likely required in the medium. They are essential for the objective and likely the database reactions have no way to make them, so gapdfilling does not wotrk.